In [1]:
import mysql.connector
import pandas as pd
from mysql.connector import Error
from faker import Faker
import numpy as np

## Introduction


In this tutorial, we will learn [how to manupulate sql databases with python](https://www.freecodecamp.org/news/connect-python-with-sql/).

In this project, we will create a database for a online internation language school for working professional. The schema of the database looks like this: 

![](https://www.freecodecamp.org/news/content/images/2020/08/ERD.png)

## Connecting to SQL server

In [18]:
## establish a connection to sql server
def connect_to_sql_server(hostname, user, password):
    
    '''
    Establish a connection between python script and MySQL server
    '''
    
    connection = None
    try:
        connection = mysql.connector.connect(
            host = hostname, 
            user = user, 
            passwd = password
        )
        print("MySQL server connection successfull")
    except Error as err: 
        print("Error in connection: ", err)
        
    return connection

In [224]:
connection = connect_to_sql_server(
    hostname = "localhost",
    user = "root",
    password = "Bora@1992"    
)
connection

MySQL server connection successfull


## Creating a new database

Now connection is successfull, we will create a new database that will store all the schema/table of the online language school:

In [24]:
def create_database(connection, db_name):
    cursor = connection.cursor()
    try:
        cursor.execute("CREATE DATABASE %s;" % db_name)
        print("Query executed successfully")
    except Error as err:
        print("Error: %s" % err)

In [225]:
connection = connect_to_sql_server(
    hostname = "localhost",
    user = "root",
    password = "Bora@1992"    
)
create_database(connection, "school")

MySQL server connection successfull
Query executed successfully


## Connecting to the database

In [2]:
def connect_to_database(hostname, user, password, db_name):
    
    connection = None
    try:
        connection = mysql.connector.connect(
            host = hostname,
            user = user,
            passwd = password,
            database = db_name
        )
        print("Connection to database: '%s' successfull."%db_name)
    except Error as err:
        print("ERROR: %s" % err)
    
    return connection

In [3]:
connection = connect_to_database(
    hostname = "localhost",
    user = "root",
    password = "Bora@1992",
    db_name = "school"
)

Connection to database: 'school' successfull.


## Creating a Query Execution Function

This function will be capable to executing any sql query:

In [207]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully.")
    except Error as err:
        print("ERROR: %s" % err)

## Creating tables

First write down the sql queries and store it in respective variables:

In [227]:
## SQL Query
create_teachers_table = '''
    CREATE TABLE teachers(
      id INT AUTO_INCREMENT PRIMARY KEY,
      first_name VARCHAR(100) NOT NULL,
      last_name VARCHAR(100) NOT NULL
    );
'''

create_students_table = '''
    CREATE TABLE students(
      id INT AUTO_INCREMENT PRIMARY KEY,
      first_name VARCHAR(100) NOT NULL,
      last_name VARCHAR(100) NOT NULL
    );
'''

create_courses_table = '''
    CREATE TABLE courses(
      id INT AUTO_INCREMENT,
      course_name VARCHAR(100) NOT NULL UNIQUE,
      teacher_id INT NOT NULL,
      FOREIGN KEY (teacher_id) REFERENCES teachers(id),
      PRIMARY KEY (id, teacher_id)
    );
'''

create_classroom_table = '''
    CREATE TABLE classroom(
        course_id INT,
        student_id INT,
        PRIMARY KEY (course_id, student_id),
        FOREIGN KEY (course_id) REFERENCES courses(id),
        FOREIGN KEY (student_id) REFERENCES students(id)
    );
'''

Now execute the sql queries:

In [228]:
## Create a mysql-python connector to the database:
connection = connect_to_database(
    hostname = "localhost",
    user = "root",
    password = "Bora@1992",
    db_name = "school"
)

execute_query(connection, create_teachers_table)
execute_query(connection, create_students_table)
execute_query(connection, create_courses_table)
execute_query(connection, create_classroom_table)

Connection to database: 'school' successfull.
Query executed successfully.
Query executed successfully.
Query executed successfully.
Query executed successfully.


## Populating tables

- first make the functions to return sql query that will create a random data for each table.

In [6]:
fake = Faker()
def teachers_insert_query(n = 10):
    
    query = "INSERT INTO teachers (first_name, last_name) VALUES"
    for _ in range(n):
        first_name, last_name = fake.name().split(" ")[0:2]
        query += "('%s','%s')," % (first_name, last_name)

    query = "%s;" % query[:-1]
    return query

def students_insert_query(n=100):
    
    query = "INSERT INTO students (first_name, last_name) VALUES"
    for _ in range(n):
        first_name, last_name = fake.name().split(" ")[0:2]
        query += "('%s','%s')," % (first_name, last_name)

    query = "%s;" % query[:-1]
    return query

def courses_insert_query(n=30, n_teachers=10):
    
    query = "INSERT INTO courses (course_name, teacher_id) VALUES"
    for _ in range(n):
        teacher_id = np.random.randint(1,n_teachers+1)
        query += "('%s', %d)," % (fake.company(), teacher_id)

    query = "%s;" % query[:-1]
    return query

def create_classroom_data(n_rows=10000, n_courses=10, n_students=100):
    combos = []
    query = "INSERT INTO classroom (course_id, student_id) VALUES "
    
    for _ in range(n_rows):
        course_id = np.random.randint(1,n_courses+1)
        student_id = np.random.randint(1,n_students+1)
        combination = (course_id,student_id)
        if combination not in combos:
            combos.append(combination)
            query += "%s," % str(combination)
        
    print("created %d entries" % len(combos))
    query = "%s;" % query[:-1]
    return query

In [230]:
n_teachers = 10
n_students = 300
n_courses = 30
n_classrooms = 10000

populate_teachers = teachers_insert_query(n_teachers)
populate_students = students_insert_query(n_students)
populate_courses = courses_insert_query(n_courses)
populate_classroom = create_classroom_data(n_rows=n_classrooms, n_courses=n_courses, n_students=n_students)

created 6101 entries


SQL queries are ready. Now we will insert the data into respective tables in the database:

In [234]:
## Create a mysql-python connector to the database:
connection = connect_to_database(
    hostname = "localhost",
    user = "root",
    password = "Bora@1992",
    db_name = "school"
)

execute_query(connection, populate_teachers)
execute_query(connection, populate_students)
execute_query(connection, populate_courses)
execute_query(connection, populate_classroom)

Connection to database: 'school' successfull.
Query executed successfully.
Query executed successfully.
Query executed successfully.
Query executed successfully.
